# Estimating Free Parking

This notebook illustrates how to re-estimate a single model component for ActivitySim.  This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [1]:
import os
import larch  # !conda install larch -c conda-forge # for estimation
import pandas as pd

We'll work in our `test` directory, where ActivitySim has saved the estimation data bundles.

In [2]:
os.chdir('test')

# Load data and prep model for estimation

In [3]:
modelname = "free_parking"

from activitysim.estimation.larch import component_model
model, data = component_model(modelname, return_data=True)

# Review data loaded from the EDB

The next step is to read the EDB, including the coefficients, model settings, utilities specification, and chooser and alternative data.

### Coefficients

In [4]:
data.coefficients

,value,constrain
coefficient_name,,
coef_asc_san_francisco,-2.6403,F
coef_asc_santa_clara,0.2118,F
coef_asc_alameda,-0.1092,F
coef_income_very_high,0.2300,F
coef_income_high,0.2300,F
coef_hh_size_4_up,0.2530,F
coef_more_autos_than_workers,0.2310,F
coef_fewer_autos_than_workers,-1.4790,F


#### Utility specification

In [5]:
data.spec

,Label,Description,Expression,free,pay
0,util_asc_san_francisco,NaN,@df.workplace_county_id == ID_SAN_FRANCISCO,coef_asc_san_francisco,0.0
1,util_asc_santa_clara,NaN,@df.workplace_county_id == ID_SANTA_CLARA,coef_asc_santa_clara,0.0
2,util_asc_alameda,NaN,@df.workplace_county_id == ID_ALAMEDA,coef_asc_alameda,0.0
3,util_income_very_high,Very high income household dummy,@df.income>=100000,coef_income_very_high,0.0
4,util_income_high,High income housheold dummy,@(df.income>=60000) & (df.income<100000),coef_income_high,0.0
5,util_hh_size_4_up,Household size is greater than 3 dummy,@df.hhsize>3,coef_hh_size_4_up,0.0
6,util_more_autos_than_workers,More automobiles than workers dummy,@df.auto_ownership>df.num_workers,coef_more_autos_than_workers,0.0
7,util_fewer_autos_than_workers,Fewer automobiles than workers dummy,@df.auto_ownership<df.num_workers,coef_fewer_autos_than_workers,0.0


### Chooser data

In [6]:
data.chooser_data

,person_id,model_choice,override_choice,util_asc_san_francisco,util_asc_santa_clara,util_asc_alameda,util_income_very_high,util_income_high,util_hh_size_4_up,util_more_autos_than_workers,...,COLLFTE,COLLPTE,TOPOLOGY,TERMINAL,household_density,employment_density,density_index,is_cbd,workplace_county_id,override_choice_code
household_id,,,,,,,,,,,,,,,,,,,,,
72241,72241,False,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.00000,1,4.58156,68.092368,151.674705,46.994710,True,1,2
72441,72441,False,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.00000,1,2.28482,28.491228,4.052632,3.547964,False,1,2
72528,72528,False,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6861.20508,1174.96875,1,3.49553,19.033424,37.235052,12.595161,False,1,2
73144,73144,False,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.00000,1,2.48345,26.073171,8.048780,6.150212,False,1,2
73493,73493,False,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7144.64307,22523.34570,1,3.20356,13.672131,39.852459,10.179771,False,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2822879,7514214,False,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.00000,1,1.08765,0.415094,3.578616,0.371951,False,1,2
2822949,7514284,False,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.00000,1,3.86254,33.841404,40.299687,18.394631,True,1,2
2823069,7514404,False,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.00000,1,3.95752,65.596535,32.655666,21.802041,True,1,2


# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has a built-in estimation methods including BHHH, and also offers access to more advanced general purpose non-linear optimizers in the `scipy` package, including SLSQP, which allows for bounds and constraints on parameters.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.

In [9]:
model.estimate()

req_data does not request avail_ca or avail_co but it is set and being provided


,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
0.0,0.000000,0.0000,0.0,0.0,0.0,1,,0.000000
coef_asc_alameda,-0.109200,-0.1092,0.0,NaN,NaN,0,,-0.109200
coef_asc_san_francisco,-2.456059,-2.6403,0.0,NaN,NaN,0,,-2.456059
coef_asc_santa_clara,0.211800,0.2118,0.0,NaN,NaN,0,,0.211800
coef_fewer_autos_than_workers,-1.619489,-1.4790,0.0,NaN,NaN,0,,-1.619489
coef_hh_size_4_up,0.279145,0.2530,0.0,NaN,NaN,0,,0.279145
coef_income_high,-0.053120,0.2300,0.0,NaN,NaN,0,,-0.053120
coef_income_very_high,0.296661,0.2300,0.0,NaN,NaN,0,,0.296661
coef_more_autos_than_workers,0.223434,0.2310,0.0,NaN,NaN,0,,0.223434


/Users/jeffnewman/OneDrive - Cambridge Systematics/Git/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 0.0 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")


┣                x: 0.0                              0.000000
┃                   coef_asc_alameda                -0.109200
┃                   coef_asc_san_francisco          -2.456059
┃                   coef_asc_santa_clara             0.211800
┃                   coef_fewer_autos_than_workers   -1.619489
┃                   coef_hh_size_4_up                0.279145
┃                   coef_income_high                -0.053120
┃                   coef_income_very_high            0.296661
┃                   coef_more_autos_than_workers     0.223434
┃                   dtype: float64
┣          loglike: -484.0908426547178
┣        d_loglike: 0.0                              0.000000
┃                   coef_asc_alameda                 0.000000
┃                   coef_asc_san_francisco           0.000399
┃                   coef_asc_santa_clara             0.000000
┃                   coef_fewer_autos_than_workers    0.000183
┃                   coef_hh_size_4_up                0.000210
┃                   coef_income_high                 0.000364
┃                   coef_income_very_high           -0.000061
┃                   coef_more_autos_than_workers     0.000107
┃                   dtype: float64
┣              nit: 10
┣             nfev: 19
┣             njev: 10
┣           status: 0
┣          message: 'Optimization terminated successfully'
┣          success: True
┣     elapsed_time: datetime.timedelta(microseconds=810470)
┣           method: 'slsqp'
┣          n_cases: 2582
┣ iteration_number: 10
┣          logloss: 0.1874867709739418

### Estimated coefficients

In [10]:
model.parameter_summary()

,Value,Std Err,t Stat,Signif,Like Ratio,Null Value,Constrained
0.0,0.00,NA,NA,,NA,0.00,fixed value
coef_asc_alameda,-0.109,0.00,NA,[],0.00,0.00,
coef_asc_san_francisco,-2.46,0.168,-14.65,***,NA,0.00,
coef_asc_santa_clara,0.212,1.43e-16,BIG,***,NA,0.00,
coef_fewer_autos_than_workers,-1.62,0.220,-7.37,***,NA,0.00,
coef_hh_size_4_up,0.279,0.196,1.43,,NA,0.00,
coef_income_high,-0.0531,0.242,-0.22,,NA,0.00,
coef_income_very_high,0.297,0.208,1.43,,NA,0.00,
coef_more_autos_than_workers,0.223,0.241,0.93,,NA,0.00,


# Output Estimation Results

In [11]:
from activitysim.estimation.larch import update_coefficients
result_dir = data.edb_directory/"estimated"
update_coefficients(
    model, data, result_dir,
    output_file=f"{modelname}_coefficients_revised.csv",
);

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [12]:
model.to_xlsx(
    result_dir/f"{modelname}_model_estimation.xlsx", 
    data_statistics=False,
)

# Next Steps

The final step is to either manually or automatically copy the `*_coefficients_revised.csv` file to the configs folder, rename it to `*_coefficients.csv`, and run ActivitySim in simulation mode.

In [13]:
pd.read_csv(result_dir/f"{modelname}_coefficients_revised.csv")

,coefficient_name,value,constrain
0,coef_asc_san_francisco,-2.456059,F
1,coef_asc_santa_clara,0.211800,F
2,coef_asc_alameda,-0.109200,F
3,coef_income_very_high,0.296661,F
4,coef_income_high,-0.053120,F
5,coef_hh_size_4_up,0.279145,F
6,coef_more_autos_than_workers,0.223434,F
7,coef_fewer_autos_than_workers,-1.619489,F
